In [3]:
import pandas as pd
import numpy as np

from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
from alphadia.extraction.utils import reannotate_fragments
from alphabase.psm_reader import psm_reader_provider

In [4]:
diann_output = '/Users/georgwallmann/Documents/data/2022_10_14_brunner_1ng_rep01/report_ms1.tsv'
insilico_lib_location = '/Users/georgwallmann/Documents/data/libraries/2022_11_29_human_trypsin_diann_decoy.hdf'
output_location = '/Users/georgwallmann/Library/CloudStorage/OneDrive-Personal/Studium/MPI/AG Mann/alphaDIA/2022_12_18_performance_testing/0_brunner_2022_1ng_extraction'

In [5]:
# create dense library from diann psm file
reader = psm_reader_provider.get_reader('diann', keep_decoy=True, fdr=1)
reader.import_file(diann_output)
raw_name = reader.psm_df.raw_name.unique()[0]
reader._psm_df = reader.psm_df[reader.psm_df.raw_name == raw_name]
speclib_psm = SpecLibBase(decoy='diann')
speclib_psm.precursor_df = reader.psm_df

# append decoys and recalculate precursor mz
speclib_psm.append_decoy_sequence()
speclib_psm.calc_precursor_mz()

# annotate library with predicted fragments
insilico_lib = SpecLibBase()
insilico_lib.load_hdf(insilico_lib_location, load_mod_seq=True)

speclib_psm.annotate_fragments_from_speclib(insilico_lib, verbose=True)
speclib_psm.remove_unused_fragments()

2022-12-22 15:15:19> Speclib with 34,741 precursors will be reannotated with speclib with 5,888,372 precursors and 394,126,900 fragments
2022-12-22 15:15:25> A total of 34,724 precursors were succesfully annotated, 17 precursors were not matched


In [6]:
speclib_psm.save_hdf(output_location + '/brunner_2022_1ng_rep01.hdf')